In [1]:
"""
https://www.youtube.com/watch?v=USTqY4gH_VM&t=285s
"""

import dash  #(version 1.12.0)
from dash.dependencies import Input, Output
import dash_table
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
import jupyter_dash

# -------------------------------------------------------------------------------------
# Import the cleaned data (importing csv into pandas)
df = pd.read_csv("internet_cleaned.csv")
df = df[df['year'] == 2019]

In [2]:
df.shape

(37, 6)

In [3]:
df.head()

,country,iso_alpha3,year,internet daily,did online course,internet at least once/week
333,Austria,AUT,2019,80.0,8.0,86.0
334,Belgium,BEL,2019,85.0,9.0,89.0
335,Bosnia and Herzegovina,BIH,2019,65.0,3.0,69.0
336,Bulgaria,BGR,2019,60.0,2.0,67.0
337,Croatia,HRV,2019,71.0,5.0,77.0


In [4]:
# Creating an ID column name gives us more interactive capabilities
df['id'] = df['iso_alpha3']
df.set_index('id', inplace=True, drop=False)

In [5]:
print(df.columns)

Index(['country', 'iso_alpha3', 'year', 'internet daily', 'did online course',
       'internet at least once/week', 'id'],
      dtype='object')


In [6]:
dict_df = df.to_dict('records')

In [7]:
dict_df[:2]

[{'country': 'Austria',
  'iso_alpha3': 'AUT',
  'year': 2019,
  'internet daily': 80.0,
  'did online course': 8.0,
  'internet at least once/week': 86.0,
  'id': 'AUT'},
 {'country': 'Belgium',
  'iso_alpha3': 'BEL',
  'year': 2019,
  'internet daily': 85.0,
  'did online course': 9.0,
  'internet at least once/week': 89.0,
  'id': 'BEL'}]

In [8]:
# -------------------------------------------------------------------------------------
# App layout
app = jupyter_dash.JupyterDash(__name__, prevent_initial_callbacks=True) # this was introduced in Dash version 1.12.0

# Sorting operators (https://dash.plotly.com/datatable/filtering)
app.layout = html.Div([
    dash_table.DataTable(
        id='datatable-interactivity',
        columns=[
            {"name": i, "id": i, "deletable": True, "selectable": True, "hideable": True}
            if i == "iso_alpha3" or i == "year" or i == "id"
            else {"name": i, "id": i, "deletable": True, "selectable": True}
            for i in df.columns
        ],
        data=df.to_dict('records'),  # the contents of the table
        editable=True,              # allow editing of data inside all cells
        filter_action="native",     # allow filtering of data by user ('native') or not ('none')
        sort_action="native",       # enables data to be sorted per-column by user or not ('none')
        sort_mode="single",         # sort across 'multi' or 'single' columns
        column_selectable="multi",  # allow users to select 'multi' or 'single' columns
        row_selectable="multi",     # allow users to select 'multi' or 'single' rows
        row_deletable=True,         # choose if user can delete a row (True) or not (False)
        selected_columns=[],        # ids of columns that user selects
        selected_rows=[],           # indices of rows that user selects
        page_action="native",       # all data is passed to the table up-front or not ('none')
        page_current=0,             # page number that user is on
        page_size=6,                # number of rows visible per page
        style_cell={                # ensure adequate header width when text is shorter than cell's text
            'minWidth': 95, 'maxWidth': 95, 'width': 95
        },
        style_cell_conditional=[    # align text columns to left. By default they are aligned to right
            {
                'if': {'column_id': c},
                'textAlign': 'left'
            } for c in ['country', 'iso_alpha3']
        ],
        style_data={                # overflow cells' content into multiple lines
            'whiteSpace': 'normal',
            'height': 'auto'
        }
    ),

    html.Br(),
    html.Br(),
    html.Div(id='bar-container'),
    html.Div(id='choromap-container')

])


# -------------------------------------------------------------------------------------
# Create bar chart
@app.callback(
    Output(component_id='bar-container', component_property='children'),
    [Input(component_id='datatable-interactivity', component_property="derived_virtual_data"),
     Input(component_id='datatable-interactivity', component_property='derived_virtual_selected_rows'),
     Input(component_id='datatable-interactivity', component_property='derived_virtual_selected_row_ids'),
     Input(component_id='datatable-interactivity', component_property='selected_rows'),
     Input(component_id='datatable-interactivity', component_property='derived_virtual_indices'),
     Input(component_id='datatable-interactivity', component_property='derived_virtual_row_ids'),
     Input(component_id='datatable-interactivity', component_property='active_cell'),
     Input(component_id='datatable-interactivity', component_property='selected_cells')]
)
def update_bar(all_rows_data, slctd_row_indices, slct_rows_names, slctd_rows,
               order_of_rows_indices, order_of_rows_names, actv_cell, slctd_cell):
    print('***************************************************************************')
    print('Data across all pages pre or post filtering: {}'.format(all_rows_data))
    print('---------------------------------------------')
    print("Indices of selected rows if part of table after filtering:{}".format(slctd_row_indices))
    print("Names of selected rows if part of table after filtering: {}".format(slct_rows_names))
    print("Indices of selected rows regardless of filtering results: {}".format(slctd_rows))
    print('---------------------------------------------')
    print("Indices of all rows pre or post filtering: {}".format(order_of_rows_indices))
    print("Names of all rows pre or post filtering: {}".format(order_of_rows_names))
    print("---------------------------------------------")
    print("Complete data of active cell: {}".format(actv_cell))
    print("Complete data of all selected cells: {}".format(slctd_cell))

    dff = pd.DataFrame(all_rows_data)

    # used to highlight selected countries on bar chart
    colors = ['#7FDBFF' if i in slctd_row_indices else '#0074D9'
              for i in range(len(dff))]

    if "country" in dff and "did online course" in dff:
        return [
            dcc.Graph(id='bar-chart',
                      figure=px.bar(
                          data_frame=dff,
                          x="country",
                          y='did online course',
                          labels={"did online course": "% of Pop took online course"}
                      ).update_layout(showlegend=False, xaxis={'categoryorder': 'total ascending'})
                      .update_traces(marker_color=colors, hovertemplate="<b>%{y}%</b><extra></extra>")
                      )
        ]


# -------------------------------------------------------------------------------------
# Create choropleth map
@app.callback(
    Output(component_id='choromap-container', component_property='children'),
    [Input(component_id='datatable-interactivity', component_property="derived_virtual_data"),
     Input(component_id='datatable-interactivity', component_property='derived_virtual_selected_rows')]
)
def update_map(all_rows_data, slctd_row_indices):
    dff = pd.DataFrame(all_rows_data)

    # highlight selected countries on map
    borders = [5 if i in slctd_row_indices else 1
               for i in range(len(dff))]

    if "iso_alpha3" in dff and "internet daily" in dff and "country" in dff:
        return [
            dcc.Graph(id='choropleth',
                      style={'height': 700},
                      figure=px.choropleth(
                          data_frame=dff,
                          locations="iso_alpha3",
                          scope="europe",
                          color="internet daily",
                          title="% of Pop that Uses Internet Daily",
                          template='plotly_dark',
                          hover_data=['country', 'internet daily'],
                      ).update_layout(showlegend=False, title=dict(font=dict(size=28), x=0.5, xanchor='center'))
                      .update_traces(marker_line_width=borders, hovertemplate="<b>%{customdata[0]}</b><br><br>" +
                                                                              "%{customdata[1]}" + "%")
                      )
        ]


# -------------------------------------------------------------------------------------
# Highlight selected column
@app.callback(
    Output('datatable-interactivity', 'style_data_conditional'),
    [Input('datatable-interactivity', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# -------------------------------------------------------------------------------------


if __name__ == '__main__':
    app.run_server(debug=True)

***************************************************************************
Data across all pages pre or post filtering: [{'country': 'Austria', 'iso_alpha3': 'AUT', 'year': 2019, 'internet daily': 80, 'did online course': 8, 'internet at least once/week': 86, 'id': 'AUT'}, {'country': 'Belgium', 'iso_alpha3': 'BEL', 'year': 2019, 'internet daily': 85, 'did online course': 9, 'internet at least once/week': 89, 'id': 'BEL'}, {'country': 'Bosnia and Herzegovina', 'iso_alpha3': 'BIH', 'year': 2019, 'internet daily': 65, 'did online course': 3, 'internet at least once/week': 69, 'id': 'BIH'}, {'country': 'Bulgaria', 'iso_alpha3': 'BGR', 'year': 2019, 'internet daily': 60, 'did online course': 2, 'internet at least once/week': 67, 'id': 'BGR'}, {'country': 'Croatia', 'iso_alpha3': 'HRV', 'year': 2019, 'internet daily': 71, 'did online course': 5, 'internet at least once/week': 77, 'id': 'HRV'}, {'country': 'Cyprus', 'iso_alpha3': 'CYP', 'year': 2019, 'internet daily': 79, 'did online course

***************************************************************************
Data across all pages pre or post filtering: [{'country': 'Austria', 'iso_alpha3': 'AUT', 'year': 2019, 'internet daily': 80, 'did online course': 8, 'internet at least once/week': 86, 'id': 'AUT'}, {'country': 'Belgium', 'iso_alpha3': 'BEL', 'year': 2019, 'internet daily': 85, 'did online course': 9, 'internet at least once/week': 89, 'id': 'BEL'}, {'country': 'Bosnia and Herzegovina', 'iso_alpha3': 'BIH', 'year': 2019, 'internet daily': 65, 'did online course': 3, 'internet at least once/week': 69, 'id': 'BIH'}, {'country': 'Bulgaria', 'iso_alpha3': 'BGR', 'year': 2019, 'internet daily': 60, 'did online course': 2, 'internet at least once/week': 67, 'id': 'BGR'}, {'country': 'Croatia', 'iso_alpha3': 'HRV', 'year': 2019, 'internet daily': 71, 'did online course': 5, 'internet at least once/week': 77, 'id': 'HRV'}, {'country': 'Cyprus', 'iso_alpha3': 'CYP', 'year': 2019, 'internet daily': 79, 'did online course

***************************************************************************
Data across all pages pre or post filtering: [{'country': 'Austria', 'iso_alpha3': 'AUT', 'year': 2019, 'internet daily': 80, 'did online course': 8, 'internet at least once/week': 86, 'id': 'AUT'}, {'country': 'Belgium', 'iso_alpha3': 'BEL', 'year': 2019, 'internet daily': 85, 'did online course': 9, 'internet at least once/week': 89, 'id': 'BEL'}, {'country': 'Bosnia and Herzegovina', 'iso_alpha3': 'BIH', 'year': 2019, 'internet daily': 65, 'did online course': 3, 'internet at least once/week': 69, 'id': 'BIH'}, {'country': 'Bulgaria', 'iso_alpha3': 'BGR', 'year': 2019, 'internet daily': 60, 'did online course': 2, 'internet at least once/week': 67, 'id': 'BGR'}, {'country': 'Croatia', 'iso_alpha3': 'HRV', 'year': 2019, 'internet daily': 71, 'did online course': 5, 'internet at least once/week': 77, 'id': 'HRV'}, {'country': 'Cyprus', 'iso_alpha3': 'CYP', 'year': 2019, 'internet daily': 79, 'did online course

***************************************************************************
Data across all pages pre or post filtering: [{'country': 'Austria', 'iso_alpha3': 'AUT', 'year': 2019, 'internet daily': 80, 'did online course': 8, 'internet at least once/week': 86, 'id': 'AUT'}, {'country': 'Belgium', 'iso_alpha3': 'BEL', 'year': 2019, 'internet daily': 85, 'did online course': 9, 'internet at least once/week': 89, 'id': 'BEL'}, {'country': 'Bosnia and Herzegovina', 'iso_alpha3': 'BIH', 'year': 2019, 'internet daily': 65, 'did online course': 3, 'internet at least once/week': 69, 'id': 'BIH'}, {'country': 'Bulgaria', 'iso_alpha3': 'BGR', 'year': 2019, 'internet daily': 60, 'did online course': 2, 'internet at least once/week': 67, 'id': 'BGR'}, {'country': 'Croatia', 'iso_alpha3': 'HRV', 'year': 2019, 'internet daily': 71, 'did online course': 5, 'internet at least once/week': 77, 'id': 'HRV'}, {'country': 'Cyprus', 'iso_alpha3': 'CYP', 'year': 2019, 'internet daily': 79, 'did online course

***************************************************************************
Data across all pages pre or post filtering: [{'country': 'Austria', 'iso_alpha3': 'AUT', 'year': 2019, 'internet daily': 80, 'did online course': 8, 'internet at least once/week': 86, 'id': 'AUT'}, {'country': 'Belgium', 'iso_alpha3': 'BEL', 'year': 2019, 'internet daily': 85, 'did online course': 9, 'internet at least once/week': 89, 'id': 'BEL'}, {'country': 'Bosnia and Herzegovina', 'iso_alpha3': 'BIH', 'year': 2019, 'internet daily': 65, 'did online course': 3, 'internet at least once/week': 69, 'id': 'BIH'}, {'country': 'Bulgaria', 'iso_alpha3': 'BGR', 'year': 2019, 'internet daily': 60, 'did online course': 2, 'internet at least once/week': 67, 'id': 'BGR'}, {'country': 'Croatia', 'iso_alpha3': 'HRV', 'year': 2019, 'internet daily': 71, 'did online course': 5, 'internet at least once/week': 77, 'id': 'HRV'}, {'country': 'Cyprus', 'iso_alpha3': 'CYP', 'year': 2019, 'internet daily': 79, 'did online course